# w261 Final Project - Clickthrough Rate Prediction

25   
Justin Trobec, Jeff Li, Sonya Chen, 
Summer 2019, section 5

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

## Explain Data

## Define Outcome

### Purpose: 
- We want to predict whether a flight will delay or not. 
- The definition of delay is that if a flight is more than 15 minutes late than its original flight time, then it is defined as late. 

### Models Evaluation and Model Metric Summary: 
- First we will use ROC / AUC to find the best model among all the possible models our team might explore. 
- After we confirm the model we want to use, then our team will use F1 score the find the best threshold to do the classification. 

### Explain Evaluation Appraoch and Core Concepts:

### Why use ROC (or AUC) to pick the best model? 
- To examine how does a classifier do, you have to examine both precision and recall rate. However, Precision and Recall are often trades off each other. Improving precision might highly reduces Recall rate. Vise versa, increasing recall rates very likely reduces precision. 
- Thus, the overall performance of a classifier, summed over all possible thresholds, is given by ROC (Receiver Operating Characteristics). 
- Also ROC can handle the scenario when the data is imbalanced.

#### What is ROC curve? 
- Is used to examined how well your classifier can separate positive and negative data points. 
- And is used to identify the best threshold for separating positive and negative examples. 
- #TODO graph / formula

#### How to use ROC? 
- To use ROC, classify need to output scores or probability. 
- ROC curve is created by plotting True positive rate (TPR) against False positive rate (FPR) at various threshold. 
- Formula or graph (#TODO)

#### What is AUC (Area Under the Curve)?
- It is the area under the ROC. 
- Best model has AUC equals to 1.0. The better the model, the higher its AUC score.
- # TODO graph / model



#### F1 score: 
- After picking the best model, Then we can use F1 score at find out which threshold has the highest F1 score. 
- F1 score is the weighted average of Precision and Recall.
- Formula: 
$$ F_1 = 2 \* \frac{\text{precision * recall}}{\text{precision + recall}} $$


#### Precision: 
- the measure of the correctly identified positive cases from all the predicted positive cases. 
- Formula: $$\text{Precision} = \frac{\text{True Positive}}{\text{True Positive + False Positive}} $$

#### Recall: 
- the measure of the correctly identified positive cases from all the actual positive cases. 
- Formula: $$ \text{Recall} = \frac{\text{True Positive}}{\text{True Positive + False Negative}} $$


sources: 
- https://medium.com/swlh/recall-precision-f1-roc-auc-and-everything-542aedf322b9
- https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/
- https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2

## Data Ingestion

## Join Datasets

## Split Train/Validation/Test

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts